In [1]:
!pip install albumentations
!pip install torchmetrics

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision.io import read_image
from torchvision.utils import draw_bounding_boxes
import cv2
import cv2 as cv
import os
import pathlib
import sys
from pathlib import Path
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import torchmetrics
from Dataloaders import supervised_loader, test_loader, supervised_train_transform, test_transform

In [4]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
import segmentation_models_pytorch as smp

model = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)

model = model.to(DEVICE)


In [6]:
LEARNING_RATE = 3e-5
BATCH_SIZE = 8
NUM_EPOCHS = 50
NUM_WORKERS = 2
IMAGE_HEIGHT = 736 
IMAGE_WIDTH = 1280 
PIN_MEMORY = True
LOAD_MODEL = False
TRAIN_IMG_DIR = "images"
TRAIN_MASK_DIR = "thims"
VAL_IMG_DIR = "val_im"
VAL_MASK_DIR = "val_lab"

In [42]:
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scaler = torch.cuda.amp.GradScaler()

In [43]:
# val_data_size = 250
# ranset = set()
# while (len(ranset) <= val_data_size):
#     ranset.add(np.random.randint(0, 2001))

In [44]:
# import shutil
# import os

# imlis = os.listdir("images")
# lablis = os.listdir("thims")

In [45]:
# for i in ranset:
#     shutil.move(f"images/{imlis[i]}", "val_im")
#     shutil.move(f"thims/{lablis[i]}", "val_lab")

In [46]:
## loading dataloaders

train_data_loader = supervised_loader(image_dir = TRAIN_IMG_DIR, mask_dir= TRAIN_MASK_DIR,
                                      transform=supervised_train_transform, batch_size=BATCH_SIZE, shuffle=True, num_workers=4,
                                      shape = (IMAGE_WIDTH, IMAGE_HEIGHT))

val_data_loader = supervised_loader(image_dir = VAL_IMG_DIR, mask_dir= VAL_MASK_DIR,
                                      transform=test_transform, batch_size=BATCH_SIZE, shuffle=True, num_workers = 4,
                                      shape = (IMAGE_WIDTH, IMAGE_HEIGHT))

In [ ]:
iou_loss_fn = torchmetrics.classification.JaccardIndex(task = 'binary', threshold = 0., num_classes = 2)

epochs = (range(NUM_EPOCHS))

val_loss_lis = []
train_loss_lis = []

train_iou = []
val_iou = []

for epoch in epochs:
    
    total_validation_loss = 0
    total_train_loss = 0
    total_train_iou = 0
    total_val_iou = 0
    
    loop = tqdm(train_data_loader)


    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        iou_train_loss = iou_loss_fn(predictions.to("cpu"), ((targets > 0)*1).to("cpu"))
        total_train_iou += iou_train_loss
        loop.set_postfix(loss=loss.item(), iou_loss = iou_train_loss)
        total_train_loss += loss.item()
    
    train_loss_lis.append(total_train_loss)
    train_iou.append(total_train_iou)

    with torch.no_grad():
        val_loop = tqdm(val_data_loader)
        for (data, targets) in val_loop:
            data = data.to(device=DEVICE)
            targets = targets.float().unsqueeze(1).to(device=DEVICE)

            predictions = model(data)
            loss = loss_fn(predictions, targets)
            total_validation_loss += loss
            iou_val_loss = iou_loss_fn(predictions.to("cpu"), ((targets > 0)*1).to("cpu"))
            total_val_iou += iou_val_loss
            val_loop.set_postfix(val_loss=loss.item(), iou_loss = iou_val_loss)
    
    val_loss_lis.append(total_validation_loss)
    val_iou.append(total_val_iou)

 32%|███▏      | 71/219 [00:53<01:49,  1.36it/s, iou_loss=tensor(0.5995), loss=-.847] 

In [ ]:
plt.plot(train_loss_lis)

In [ ]:
plt.plot(train_loss_lis)

In [ ]:
plt.plot(val_loss_lis)

In [ ]:
plt.plot(val_iou)

In [ ]:
plt.plot(train_iou)

In [ ]:
torch.save(model.state_dict(), f"unet1750_epoch")